In [2]:
import os

import torch
from pytorch_lightning import seed_everything

In [3]:
from bci_aic3.models.eegnet import EEGNet
from bci_aic3.paths import (
    MI_CONFIG_PATH,
    MI_RUNS_DIR,
    RAW_DATA_DIR,
    SSVEP_CONFIG_PATH,
    SSVEP_RUNS_DIR
)
from bci_aic3.train import train_model
from bci_aic3.util import save_model, load_model

In [ ]:

# Code necessary to create reproducible runs
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
seed_everything(42, workers=True)
torch.use_deterministic_algorithms(True, warn_only=True)


In [ ]:
trainer, model = train_model(model=EEGNet, config_path=MI_CONFIG_PATH)

In [ ]:
from bci_aic3.inference import (
    create_inference_data_loader,
    load_and_preprocess_for_inference,
    predict_batch,
)


In [ ]:
test_dataset = load_and_preprocess_for_inference(
    "test.csv", base_path=RAW_DATA_DIR, task_type="SSVEP"
)

test_loader = create_inference_data_loader(test_dataset)